# Visualize solutions

2022.11.07

In [9]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
import joptimise
using Printf
using JSON

In [2]:
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyKaleido not found in current path:\n- Run `import Pkg; Pkg.add(\"PlotlyKaleido\")` to install the PlotlyKaleido package.\n")
└ @ Plots C:\Users\yshimane3\.julia\packages\Plots\fw4rv\src\backends.jl:426


Plots.PlotlyBackend()

In [3]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

Main.SailorMoon

In [4]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [5]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")
res.flag

Halo guess Az_km: 1200.0

1

In [6]:
x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

Linear stability ν = 618.7618470919092


In [7]:
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12
);

In [63]:
tmax_si = 0.3  # N
isp_si = 3500  # sec
mdot_si = tmax_si / (isp_si * 9.81)
mstar = 2000  # kg
rp_parking = (6378+200)/param3b.lstar   # parking orbit radius

tmax = AstrodynamicsBase.dimensional2canonical_thrust(
    tmax_si, mstar, param3b.lstar, param3b.tstar
)
mdot = AstrodynamicsBase.dimensional2canonical_mdot(
    mdot_si, mstar, param3b.tstar
)

0.0016413295927890758

In [64]:
propagate_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false) where T
    # unpack
    nx = length(x)
    θf, tof, eta, r_apogee, ecc, raan, ϕ, m0, mf = x[1:9]  # θf: Sun angle at final time
    tau1     = x[10 : floor(Int, (nx-9)/2 + 9)]  # discretization numbers are the same for the first & second arc
    tau2     = x[floor(Int, (nx-9)/2 + 10) : end]
    tof_fwd = tof * eta
    tof_bck = tof * (1 - eta)

    # construct initial state
    sma = (rp_parking + r_apogee)/2
    sv0_kep = [sma, ecc, 0.0, raan, 0.0, 0.0]
    θ0 = θf - param3b.oms*(tof_fwd + tof_bck)   # initial Sun angle
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
    sv0 = vcat(inertial2rotating(sv0_i, θ0, 1.0) + [-param3b.mu2,0,0,0,0,0], m0)

    # construct final state
    #x0_stm = vcat(LPOArrival.x0, reshape(I(6), (36,)))
    #tspan = [0, ϕ*LPOArrival.period]
    #prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, tspan, [param3b.mu2,])
    #sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)
    svf = vcat(SailorMoon.set_terminal_state(ϕ, param3b, LPOArrival), mf)

    # initialize storage
    sols_fwd, sols_bck = [], []
    params_fwd, params_bck = [], []

    # forward propagation
    nhalf = Int(n/2)
    tspan_fwd = [0, tof_fwd/nhalf]
    for i = 1:nhalf
        τ, γ, β = tau1[3*i-2 : 3*i]
        params = [param3b.mu2, param3b.mus, θ0, param3b.as, param3b.oms, τ, γ, β, mdot, tmax]
        _prob = remake(
            _prob_base; tspan=tspan_fwd,
            u0 = sv0,
            p = params,
        )
        sol = DifferentialEquations.solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
        if get_sols
            push!(sols_fwd, sol)
            push!(params_fwd, params)
        end
        # update θ0 and sv0
        θ0 += param3b.oms*sol.t[end]
        sv0 = sol.u[end]
    end

    # back propagation
    tspan_bck = [0, -tof_bck/(n-nhalf)]
    for i = 1:n-nhalf
        τ, γ, β = tau2[3*i-2 : 3*i]
        params = [param3b.mu2, param3b.mus, θf, param3b.as, param3b.oms, τ, γ, β, mdot, tmax]
        _prob = remake(
            _prob_base; tspan=tspan_bck,
            u0 = svf,
            p = params,
        )
        sol = DifferentialEquations.solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
        if get_sols
            push!(sols_bck, sol)
            push!(params_bck, params)
        end
        # update θf (note sol.t[end] is negative so += is correct) and svf
        θf += param3b.oms*sol.t[end]
        svf = sol.u[end]
    end

    # residual
    if get_sols == false
        return svf - sv0
    else
        return sols_fwd, sols_bck, params_fwd, params_bck
    end
end

#19 (generic function with 2 methods)

## Load solution

In [65]:
function xprint(x)
    θf, tof, eta, sma, ecc, raan, ϕ, m0, mf = x[1:9]
    @printf("Launch SMA  : %1.4f\n", sma)
    @printf("Launch ECC  : %1.4f\n", ecc)
    @printf("Launch RAAN : %3.4f\n", rad2deg(raan))
    @printf("TOF [day]   : %3.4f\n", tof*param3b.tstar/86400)
    @printf("TOF [TU]    : %3.4f\n", tof)
    @printf("m0          : %2.4f\n", m0)
    @printf("mf          : %2.4f\n", mf)
end

function get_controls(x)
    nx = length(x)
    tau1     = x[10 : floor(Int, (nx-9)/2 + 9)]  # discretization numbers are the same for the first & second arc
    tau2     = x[floor(Int, (nx-9)/2 + 10) : end]
    tau1_list = [tau1[3*i-2 : 3*i] for i = 1:Int(n/2)]
    tau2_list = [tau2[3*i-2 : 3*i] for i = 1:n-Int(n/2)]
    tau1 = hcat(tau1_list...)
    tau2 = hcat(tau2_list...)
    return tau1, tau2
end


get_controls (generic function with 1 method)

In [66]:
sol_dict = JSON.parsefile(joinpath("..", "results", "test_save_minm0.json"))
#sol_dict = JSON.parsefile(joinpath("..", "results", "test_save_mintof.json"))

Dict{String, Any} with 6 entries:
  "lpo_period" => 3.41036
  "tmax"       => 0.0550298
  "lpo_x0"     => Any[1.12269, 0.0, -5.22889e-26, 0.0, 0.165099, 0.0]
  "xopt"       => Any[3.00964, 19.7989, 0.45264, 2.23055, 0.929739, 1.30066, 0.…
  "mdot"       => 0.00164133
  "n"          => 20

In [67]:
n = sol_dict["n"]
xopt = [el for el in sol_dict["xopt"]];
params = [param3b.mu2, param3b.mus, 0.0, param3b.as, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
_prob_base = ODEProblem(R3BP.rhs_bcr4bp_thrust!, rand(7), [0,1], params)


sols_fwd, sols_bck, params_fwd, params_bck = propagate_trajectory(xopt, true);
propagate_trajectory(xopt, false)

7-element Vector{Float64}:
  2.1662503714594146
 -0.6440965317737153
 -2.0555652350043086e-25
 -0.6604910024854904
 -2.078213957023937
 -7.416913824812092e-27
  0.0

In [68]:
xprint(xopt)

Launch SMA  : 2.2306
Launch ECC  : 0.9297
Launch RAAN : 74.5224
TOF [day]   : 86.0932
TOF [TU]    : 19.7989
m0          : 1.0000
mf          : 1.0000


In [69]:
rp_parking = (6378+200)/param3b.lstar
r_apogee = 1.71e6/param3b.lstar
(rp_parking + r_apogee)/2, r_apogee

(2.2307803539321944, 4.444463817227126)

In [70]:
# plot trajectory
cs_fwd = palette([:darkred, :navyblue], max(Int(n/2),2))
cs_bck = palette([:darkgreen, :goldenrod], max(n-Int(n/2),2))

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
for (ifwd,sol_fwd) in enumerate(sols_fwd)
    plot!(pcart, Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=cs_fwd[ifwd],
        linewidth=1.5, label="fwd $ifwd", linestyle=:dashdot)
end
for (ibck,sol_bck) in enumerate(sols_bck)
    plot!(pcart, Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=cs_bck[ibck],
        linewidth=1.5, label="bck $ibck", linestyle=:solid)
end
plot!(pcart; title="Initial guess")
pcart

In [71]:
get_bc_states = function (x::AbstractVector{T}) where T
    # unpack
    nx = length(x)
    θf, tof, eta, sma, ecc, raan, ϕ, m0, mf = x[1:9]  # θf: Sun angle at final time
    tau1     = x[10 : floor(Int, (nx-9)/2 + 9)]  # discretization numbers are the same for the first & second arc
    tau2     = x[floor(Int, (nx-9)/2 + 10) : end]
    tof_fwd = tof * eta
    tof_bck = tof * (1 - eta)

    # construct initial state
    sv0_kep = [sma, ecc, 0.0, raan, 0.0, 0.0]
    θ0 = θf - param3b.oms*(tof_fwd + tof_bck)   # initial Sun angle
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
    sv0 = vcat(inertial2rotating(sv0_i, θ0, 1.0) + [-param3b.mu2,0,0,0,0,0], m0)

    # construct final state
    #x0_stm = vcat(LPOArrival.x0, reshape(I(6), (36,)))
    #tspan = [0, ϕ*LPOArrival.period]
    #prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, tspan, [param3b.mu2,])
    #sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)
    svf = vcat(SailorMoon.set_terminal_state(ϕ, param3b, LPOArrival), mf)
    return sv0, svf
end

#25 (generic function with 1 method)

In [72]:
sv0, svf = get_bc_states(xopt);
norm(sv0[1:3] - [-param3b.mu2, 0, 0]) * param3b.lstar

60298.04856547955

In [73]:
rp_parking = (6378+200)/param3b.lstar

0.01709689063726318